In [27]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


In [50]:
class NeuralNet:


    def __init__(self, layers=[], learning_rate=0.001, iterations=100):
        self.params = {}
        self.learning_rate = learning_rate
        self.iterations = iterations
        self.loss = []
        self.sample_size = None
        self.layers = layers
        self.X = None
        self.y = None
      
    def init_weights(self):

        np.random.seed(1) # Seed the random number generator
        for i in range(1, len(self.layers)):
          self.params["W" + str(i)] = np.random.randn(self.layers[i-1], self.layers[i]) 
          self.params['b' + str(i)]  = np.random.randn(self.layers[i],)
       # print(self.params)

    def relu(self,Z):
        return np.maximum(0,Z)

    def dRelu(self, x):
        x[x<=0] = 0
        x[x>0] = 1
        return x

    def eta(self, x):
        ETA = 0.0000000001
        return np.maximum(x, ETA)

    def sigmoid(self,Z):
        return 1/(1+np.exp(-Z))

    def mse(self,y, yhat):
        nsample = len(y)
        yhat_inv = 1.0 - yhat
        y_inv = 1.0 - y
        yhat = self.eta(yhat) ## clips value to avoid NaNs in log
        yhat_inv = self.eta(yhat_inv) 
        loss = -1/nsample * (np.sum(np.multiply(np.log(yhat), y) + np.multiply((y_inv), np.log(yhat_inv))))
        return loss

    def forward_propagation(self):

        A = self.X
        for i in range(1, len(self.layers)):
            d = np.dot(A, self.params['W'+str(i)])
            Z = d + self.params['b'+str(i)]
            if i == len(self.layers)-1:
                A = self.sigmoid(Z)
            else:
                pass
        
        yhat = A
        loss = self.mse(self.y,yhat)

        # save calculated parameters     
        self.params['A'] = A

        return yhat,loss

    def back_propagation(self,yhat):

        A = self.params['A']
        L = len(self.layers) - 1
        y_inv = 1 - self.y
        yhat_inv = 1 - yhat


        dl_wrt_yhat = np.divide(y_inv, self.eta(yhat_inv)) - np.divide(self.y, self.eta(yhat))
        dl_wrt_sig = yhat * (yhat_inv)
        dl_wrt_z = [None] * L
        dl_wrt_A = [None] * L
        dl_wrt_w = [None] * L
        dl_wrt_b = [None] * L

        dl_wrt_z[L-1] = dl_wrt_yhat * dl_wrt_sig
        dl_wrt_A[L-1] = np.dot(dl_wrt_z[L-1], self.params['W'+str(L)].T)
        dl_wrt_w[L-1] = np.dot(A.T, dl_wrt_z[L-1])
        dl_wrt_b[L-1] = np.sum(dl_wrt_z[L-1], axis=0)

        for l in range(L-2, -1, -1):
            dl_wrt_z[l] = dl_wrt_A[l+1] * self.dRelu(A)
            dl_wrt_A[l] = np.dot(dl_wrt_z[l], self.params['W'+str(l+1)].T)
            dl_wrt_w[l] = np.dot(self.params['A'+str(l)].T, dl_wrt_z[l])
            dl_wrt_b[l] = np.sum(dl_wrt_z[l], axis=0)
            
        for i in range(1, L):
            self.params['W'+str(i)] -= self.learning_rate * dl_wrt_w[i-1]
            self.params['b'+str(i)] -= self.learning_rate * dl_wrt_b[i-1]
            
    def fit(self, X, y):

        self.X = X
        self.y = y
        self.init_weights() # initialize weights and bias

        for i in range(self.iterations):
            yhat, loss = self.forward_propagation()
            self.back_propagation(yhat)
            self.loss.append(loss)
        print(len(yhat))
    
    
    def predict(self, X):

        A = X
        for i in range(1, len(self.layers)):
          Z = A.dot(self.params['W'+str(i)]) + self.params['b'+str(i)]
        
          if i == len(self.layers)-1:
            A = self.sigmoid(Z)
            A = self.relu(Z)
        return np.round(A)




In [51]:
boston = load_boston()
X = boston.data[:,0:13]
y = boston.target

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np


        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the California housing dataset (i.e.

In [52]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 25)

In [53]:
input = len(X[0]) # no. of features
output = len(y) #no. of classes

layers = [input, 3, 8, output] # no. of nodes 
model = NeuralNet(layers = layers)

In [54]:
model.fit(X_train, y_train)

ValueError: ignored

In [ ]:
from sklearn.metrics import r2_score

# Predicting R2 Score the Train set results
y_pred = model.predict(X_train)
r2_score = r2_score(y_train, y_pred)

# Predicting R2 Score the Test set results
y_pred_test = model.predict(X_test)
r2_score = r2_score(y_test, y_pred_test)

ValueError: ignored

In [ ]:
output

506